In [1]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import pyspark.sql.functions as func
import time

# Function for Changing Data Type

In [2]:
##Function for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

# UDF for Bucketing Fields from HOTEL_DIM Table

In [3]:
#Star Rating Buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'
    
#City Buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city in ('TORONTO' , 'NIAGARA FALLS' , 'MONTREAL' , 'VANCOUVER' , 'QUEBEC' , 'BANFF' , 'OTTAWA' , 'EDMONTON' , 'VICTORIA','CALGARY' , 'WHISTLER'):
			return city
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'
    

In [4]:
#udf for hotel type 
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' , 'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE') :
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'APART-HOTEL') :
			return Hotel_type
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type in ('HOTEL' , 'MOTEL') :
			return Hotel_type
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			return 'others'
	else:
		return 'NOT APPLICABLE'

In [5]:
##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market in  ('TORONTO, ON, CAN' , 'NIAGARA FALLS, ON, CAN' , 'MONTREAL, QC, CAN' , 'BANFF AREA, AB, CAN' ,'VANCOUVER, BC, CAN' ,'QUEBEC CITY, QC, CAN' , 'EDMONTON, AB, CAN' , 'OTTAWA, ON, CAN' , 'OKANAGAN VALLEY, BC, CAN'):
			return market
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN','TOFINO - UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' , 'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market in  ('MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM' , 'MEXICO - CANCUN & ISLA MUJERES' ,'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' , 'MEXICO - LOS CABOS' , 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' ,'CENTRAL AMERICA - BELIZE' , 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' , 'CENTRAL AMERICA - COSTA RICA - OTHER' , 'MEXICO - COZUMEL''MEXICO - HUATULCO' , 'MEXICO - IXTAPA & ZIHUATANEJO' , 'CENTRAL AMERICA - HONDURAS' , 'CENTRAL AMERICA - PANAMA - OTHER') :
			return market
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		elif market in ('CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR',                        'MEXICO - ACAPULCO'):
			return 'lclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market in ('PUNTA CANA, DOMINICAN REPUBLIC' , 'TURKS AND CAICOS' , 'MONTEGO BAY, JAMAICA' , 'ST. LUCIA' ,'OCHO RIOS, JAMAICA' ,                      'NEGRIL, JAMAICA' , 'BARBADOS'):
			return market
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in (  'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN','ONTARIO WEST, ON, CAN','ABBOTSFORD - CHILLIWACK, BC, CAN',                       'MAINE - COAST','ONTARIO EAST, ON, CAN','VANCOUVER ISLAND, BC, CAN','BURLINGTON, VT','CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ('ALBERTA EAST, AB, CAN','REGINA, SK, CAN','KINGSTON, ON, CAN','SASKATOON, SK, CAN','MONCTON, NB, CAN',                        'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN','RED DEER, AB, CAN','BRITISH COLUMBIA SOUTH, BC, CAN','GRAND FORKS, ND','SUDBURY, ON, CAN',                        'ONTARIO NORTH, ON, CAN','BANGOR, ME','HAMILTON - BRANTFORD, ON, CAN','BELLEVILLE - TRENTON - COBOURG, ON, CAN','FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY','PARKSVILLE, VANCOUVER ISLAND, BC, CAN','LAKE GEORGE, NY','OGUNQUIT, ME','EVERETT, WA',                        'COASTAL, BC, CAN','GROVE CITY, PA','BELLINGHAM, WA','NORTH CONWAY, NH','NEWFOUNDLAND OTHER, NL, CAN',                        'NOVA SCOTIA SOUTH, NS, CAN','PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN','FREDERICTON, NB, CAN','BRANDON, MB, CAN','ERIE, PA','NANAIMO, VANCOUVER ISLAND, BC, CAN',                        'SASKATCHEWAN SOUTH, SK, CAN','PRINCE GEORGE - QUESNEL, BC, CAN','MEDICINE HAT, AB, CAN','GRANDE PRAIRIE, AB, CAN',                        'ALBERTA NORTH, AB, CAN','DURHAM, ON, CAN','BRITISH COLUMBIA CENTRAL, BC, CAN','ONTARIO SOUTHWEST, ON, CAN',                        'SAULT STE. MARIE, ON, CAN','THUNDER BAY, ON, CAN','SAINT JOHN, NB, CAN','GREAT FALLS, MT','MINOT, ND (AREA)',                        'BRITISH COLUMBIA NORTH, BC, CAN','NORTH BAY, ON, CAN','PETERBOROUGH, ON, CAN','CAPE BRETON ISLAND, NS, CAN','ALBERTA WEST, AB, CAN',                        'HUDSON VALLEY - POUGHKEEPSIE, NY','FORT MCMURRAY, AB, CAN','NOVA SCOTIA NORTH, NS, CAN','WATERTOWN, NY','EDMUNSTON, NB, CAN',                        'OLYMPIC NATIONAL PARK AREA, WA','CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT','SUMMERSIDE, PEI, CAN','SALEM, OR (AREA)','BUTTE - HELENA, MT'                        ,'BECKLEY, WV'): 
			return 'G5'
		elif market in ('SAGINAW, MI','WHITE MOUNTAINS, NH','FINGER LAKES NEW YORK','NEW YORK - WEST','NEW BRUNSWICK NORTH, NB, CAN',                        'SASKATCHEWAN NORTH, SK, CAN','MANITOBA SOUTH, MB, CAN','BAR HARBOR, ME','WASHINGTON NORTHWEST','VERMONT CENTRAL','ALBERTA SOUTH, AB, CAN',                        'WENATCHEE-LEAVENWORTH,WA','GANANOQUE, ON, CAN','PLATTSBURGH, (NY)','STRAITS OF MACKINAC, MI','SANDUSKY, OH','MANCHESTER, NH','ITHACA, NY',                        'MINNESOTA NORTHWEST','MISSOULA, MT (AREA)','YELLOWKNIFE, NT, CAN','PEI OTHER, PEI, CAN','BILLINGS, MT (AREA)','RAPID CITY - MOUNT RUSHMORE, SD',                        'ANDOVER, MA','YUMA, AZ','PORT HURON, MI','LANCASTER, PA','FORT LEE - PARAMUS, NJ','LEXINGTON, KY','MANITOBA NORTH, MB, CAN','ABERDEEN - OCEAN SHORES, WA',                        'CANADA TERRITORIES, YT, CAN','KNOXVILLE, TN','BURLINGTON - MOUNT VERNON, WA','FREDERICKSBURG, VA','ARIZONA SOUTHEAST',                        'WINDHAM COUNTY, VT','KENAI PENINSULA, AK','COLUMBIA, SC','CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY','SANDPOINT, ID (AREA)','MOAB - GREEN RIVER, UT','WHITEHORSE, YT, CAN','NEW YORK - NORTHEAST',                         'SARNIA, ON, CAN','AUGUSTA, ME','LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ','NEW YORK - SOUTH','BISMARK, ND',                         'ALABAMA NORTH','ASHLAND - MEDFORD, OR','WASHINGTON NORTHEAST','THE BERKSHIRES, MA','EUGENE, OR (AREA)',                         'GRAND RAPIDS, MI','WASHINGTON SOUTHWEST','LAKE HAVASU CITY, AZ','SARATOGA, NY','WYTHEVILLE - VA',                         'ARCATA - EUREKA, CA','SOUTH CAROLINA EAST','CHATTANOOGA, TN','TOLEDO, OH','IDAHO FALLS, ID','WEST VIRGINIA SOUTH',                         'SPRINGFIELD, MA','OLYMPIA, WA','GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'

In [6]:
## UDF for Model & Region:
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)',                    'MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
    
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "Other Region"
    
def srch_region_bucket(region,model_name):
	if model_name == 'Other Region':
		if region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS' , 'MID-ATLANTIC' , 'GREECE & TURKEY' , 'OCEANIA' , 'ITALY NORTH' ,                      'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN'):
			return region
		elif region in ('PARIS' , 'LONDON' , 'ENGLAND'):
			return 'G9'
		elif region in ('EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND'):
			return 'G10'
		elif region in ('SOUTH AMERICA' , 'AFRICA AND EAST MED'):
			return 'G11'
		elif region in ('EASTERN EUROPE' , 'GERMANY, AUSTRIA & SWITZERLAND' , 'BENELUX' , 'FRANCE' , 'NORDIC'):
			return 'G12'
		elif region in ('JAPAN & MICRONESIA' , 'HONG KONG & MACAU' , 'SOUTHEAST ASIA EMERGING' , 'MAINLAND CHINA'):
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'

In [7]:
## Initializing all the UDFs

udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

# Importing Hotel Dimension Table

In [8]:
## Loading Hotel Dim table:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = (sqlCtx.read.format("com.databricks.spark.csv")
            .options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N')
            .load(path_hotel_dim)
            .distinct()
            .filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'")
            .select("C0","C7","C25","C26","C4","C13","C3")
            .withColumnRenamed("C0","HOTEL_ID")
            .withColumnRenamed("C7" , "city")
            .withColumnRenamed("C25","region")
            .withColumnRenamed("C26","market")
            .withColumnRenamed("C4","brand")
            .withColumnRenamed("C13","star_rating")
            .withColumnRenamed("C3","hotel_type")
            .filter("HOTEL_ID!='None'")
            )               


In [9]:
## Finalizing Hotel Dim Table
hotel_dim_with_bkts = (hotel_dim.withColumn("MODEL_NAME", udfsrch_model_name("region"))
                        .withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating"))
                        .withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME"))
                        .withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME"))
                        .withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME"))
                        .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME")))

hotel_dim_final = (hotel_dim_with_bkts
                   .drop('city')
                   .drop('market')
                   .drop('brand')
                   .drop('star_rating')
                   .drop('hotel_type')
                   .repartition(1)
                   .cache())

# UDF for  Bucketing LOS and BW from Hotel Click Book Agg Table

In [10]:
##los bucket for search los
def searchToCategory(srch_los,):
    if srch_los in (0,1,2,3,4,5,6,7):
        return str(srch_los)
    elif srch_los >= 8 and srch_los <= 14:
        return '8-14'
    else:
        return '>14'

##bw bucket for srch bw 
def searchbucketToCategory(SRCH_WINDOW):
    if SRCH_WINDOW <= 0:
        return '<=0'
    elif SRCH_WINDOW == 1:
        return '1'
    elif SRCH_WINDOW == 2:
        return '2'
    elif SRCH_WINDOW >= 3 and SRCH_WINDOW <= 4:
        return '3-4'
    elif SRCH_WINDOW >= 5 and SRCH_WINDOW <= 7:
        return '5-7'
    elif SRCH_WINDOW >= 8 and SRCH_WINDOW <= 14:
        return '8-14'
    elif SRCH_WINDOW >= 15 and SRCH_WINDOW <= 20:
        return '15-20'
    elif SRCH_WINDOW >= 21 and SRCH_WINDOW <= 30:
        return '21-30'
    elif SRCH_WINDOW >= 31 and SRCH_WINDOW <= 40:
        return '31-40'
    elif SRCH_WINDOW >= 41 and SRCH_WINDOW <= 60:
        return '41-60'
    elif SRCH_WINDOW >= 61 and SRCH_WINDOW <= 90:
        return '61-90'
    elif SRCH_WINDOW >= 91 and SRCH_WINDOW <= 120:
        return '91-120'
    elif SRCH_WINDOW >= 121 and SRCH_WINDOW <= 180:
        return '121-180'
    elif SRCH_WINDOW >= 181 and SRCH_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'
    


In [11]:
#los bucket for booked los
def srchBookedLosBucket(booked_los):
    if booked_los is None:
        return '0'
    elif booked_los  in (0,1,2,3,4,5,6,7):
        return str(booked_los)
    elif booked_los >= 8 and booked_los <= 14:
        return '8-14'
    else:
        return '>14'

#bw bucket for booked bw
def srchBookedBwBucket(BOOKED_BKG_WINDOW):
    if BOOKED_BKG_WINDOW is None:
        return '0'
    elif BOOKED_BKG_WINDOW <= 0:
        return '<= 0'
    elif BOOKED_BKG_WINDOW == 1:
        return '1'
    elif BOOKED_BKG_WINDOW == 2:
        return '2'
    elif BOOKED_BKG_WINDOW >= 3 and BOOKED_BKG_WINDOW <= 4:
        return '3-4'
    elif BOOKED_BKG_WINDOW >= 5 and BOOKED_BKG_WINDOW <= 7:
        return '5-7'
    elif BOOKED_BKG_WINDOW >= 8 and BOOKED_BKG_WINDOW <= 14:
        return '8-14'
    elif BOOKED_BKG_WINDOW >= 15 and BOOKED_BKG_WINDOW <= 20:
        return '15-20'
    elif BOOKED_BKG_WINDOW >= 21 and BOOKED_BKG_WINDOW <= 30:
        return '21-30'
    elif BOOKED_BKG_WINDOW >= 31 and BOOKED_BKG_WINDOW <= 40:
        return '31-40'
    elif BOOKED_BKG_WINDOW >= 41 and BOOKED_BKG_WINDOW <= 60:
        return '41-60'
    elif BOOKED_BKG_WINDOW >= 61 and BOOKED_BKG_WINDOW <= 90:
        return '61-90'
    elif BOOKED_BKG_WINDOW >= 91 and BOOKED_BKG_WINDOW <= 120:
        return '91-120'
    elif BOOKED_BKG_WINDOW >= 121 and BOOKED_BKG_WINDOW <= 180:
        return '121-180'
    elif BOOKED_BKG_WINDOW >= 181 and BOOKED_BKG_WINDOW <= 240:
        return '181-240'
    else:
        return '>240'

In [12]:
#converting booking flag from string values to boolean
def booking_flag(booked_flag):
    if booked_flag == 'Y':
        return 1
    else:
        return 0

In [13]:
# Registering Functions as UDF's
udfsearchToCategory=udf(searchToCategory, StringType())
udfsearchbucketToCategory=udf(searchbucketToCategory, StringType())
udfsrchBookedLosBucket=udf(srchBookedLosBucket, StringType())
udfsrchsrchBookedBwBucket=udf(srchBookedBwBucket, StringType())
booking_flag_udf=udf(booking_flag, IntegerType())

# Importing Hotel Click Book Agg Table

In [14]:
from datetime import *
import time

date_list = ["2016-05-29",
"2016-05-30",
"2016-05-31",
"2016-06-01",
"2016-06-02",
"2016-06-03",
"2016-06-04",
"2016-06-05",
"2016-06-06",
"2016-06-07",
"2016-06-08",
"2016-06-09",
"2016-06-10",
"2016-06-11",
"2016-06-12",
"2016-06-13",
"2016-06-14",
"2016-06-15",
"2016-06-16",
"2016-06-17",
"2016-06-18",
"2016-06-19",
"2016-06-20",
"2016-06-21",
"2016-06-22",
"2016-06-23",
"2016-06-24",
"2016-06-25",
"2015-06-21",
"2015-06-22",
"2015-06-23",
"2015-06-24",
"2015-06-25",
"2015-06-26",
"2015-06-27"]

start_date = datetime.strptime("2016-05-28", "%Y-%m-%d")

for i in range(91):
    x = (start_date - timedelta(i))
    x = x.strftime("%Y-%m-%d")
    date_list.append(x)

    
#schema for HCBA table
hcba_schema = StructType([
                            StructField('PARTNER_POS',StringType(), True),
                            StructField('DEVICE_TYPE',StringType(), True),
                            StructField('CLICKED_HOTEL_ID',IntegerType(), True),
                            StructField('SRCH_WINDOW',IntegerType(), True),
                            StructField('SRCH_LOS',IntegerType(), True),
                            StructField('TRAFFIC',IntegerType(), True),
                            StructField('COST',DoubleType(), True),
                            StructField('BOOKED_FLAG',StringType(), True),
                            StructField('BOOKED_HOTEL_ID',IntegerType(), True),
                            StructField('BOOKED_LOS',IntegerType(), True),
                            StructField('BOOKED_BKG_WINDOW',IntegerType(), True),
                            StructField('GROSS_PROFIT',DoubleType(), True),
                            StructField('PARTNER_ORG',StringType(), True),
                            StructField('ACTIVITY_DATE',StringType(), True)
                          ])

In [15]:
##CREATE EMPTY DATAFRAME 
hcba = sqlContext.createDataFrame(sc.emptyRDD(), hcba_schema)

In [16]:
##APPENDING THE DATA IN ABOVE CREATED hcba TABLE 
for dates in date_list:
    path_hcba = 's3n://ewe-core-meta-prod/CORE/HOTEL_CLICK_BOOK_AGG/local_date='+dates+'/*'  
    
    try :
        temp_hcba = (sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferschema = True, delimiter = "\t",nullValue= '\\N')
                    .load(path_hcba)
                    .select('C1','C3','C7','C8','C9','C13','C14','C19','C20','C21','C22','C45','C65')
                    .filter("C65 = 'TRIPADVISOR' and C1 = 'CA' and C7 != '0' and C13 != '0'") 
                    .withColumn('ACTIVITY_DATE',lit(dates))
                    )
        hcba = hcba.unionAll(temp_hcba)
        
    except: 
        print (path_hcba)
        print('file does not exist')

# Cleaning and Outlier Treatment of HCBA Data

In [17]:
## Changing Data Type of HCBA Table
cols_int = ['SRCH_LOS','SRCH_WINDOW','TRAFFIC','BOOKED_BKG_WINDOW','BOOKED_LOS']
cols_float = ['COST','GROSS_PROFIT']

hcba_changed_data_type = data_type_change_float(data_type_change_int(hcba,cols_int),cols_float)

hcba_final = (hcba_changed_data_type
                .withColumn("SRCH_LOS_BUCKET", udfsearchToCategory("SRCH_LOS"))
                .withColumn("SRCH_BW_BUCKET", udfsearchbucketToCategory("SRCH_WINDOW"))
                .withColumn("BOOKED_LOS_BUCKET", udfsrchBookedLosBucket("BOOKED_LOS"))
                .withColumn("BOOKED_BW_BUCKET", udfsrchsrchBookedBwBucket("BOOKED_BKG_WINDOW"))
                .withColumn("BOOKING", booking_flag_udf("BOOKED_FLAG"))
                .repartition(1)
             )


In [18]:
#columns required in final cleaned hcba table
cols_required = ['PARTNER_POS','DEVICE_TYPE','HOTEL_ID','PARTNER_ORG','ACTIVITY_DATE','TRAFFIC','BOOKING_WINDOW','LOS',
                 'COST','GROSS_PROFIT','LOS_BUCKET','BW_BUCKET','BOOKING']

In [19]:
#replacing the wrongly populated search parameters for the search queries that are booked with booking parameters
hcba_replac_srch_with_booking = (hcba_final.filter("BOOKED_FLAG = 'Y'")
                                 .filter("SRCH_WINDOW < -1 or SRCH_LOS <= 0")
                                 .withColumnRenamed('BOOKED_BKG_WINDOW', 'BOOKING_WINDOW')
                                 .withColumnRenamed('BOOKED_BW_BUCKET', 'BW_BUCKET')
                                 .withColumnRenamed('BOOKED_LOS_BUCKET', 'LOS_BUCKET')
                                 .withColumnRenamed('BOOKED_HOTEL_ID', 'HOTEL_ID')
                                 .withColumnRenamed('BOOKED_LOS', 'LOS')
                                 .select(cols_required)
                            )

In [20]:
#filtering out the outliers from the seacrh data
hcba_srch_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'N' and SRCH_WINDOW >= -1 and SRCH_LOS > 0 and GROSS_PROFIT = 0.0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )


In [21]:
#filtering out the outilers for the booking data
hcba_booking_data_cleaned = (hcba_final.filter("BOOKED_FLAG= 'Y' and SRCH_WINDOW >= -1 and SRCH_LOS > 0")
                             .withColumnRenamed('CLICKED_HOTEL_ID', 'HOTEL_ID')
                             .withColumnRenamed('SRCH_WINDOW', 'BOOKING_WINDOW')
                             .withColumnRenamed('SRCH_BW_BUCKET', 'BW_BUCKET')
                             .withColumnRenamed('SRCH_LOS', 'LOS')
                             .withColumnRenamed('SRCH_LOS_BUCKET', 'LOS_BUCKET')
                             .select(cols_required)
                            )

In [22]:
#Union of all the above 3 tables 
hcba_cleaned_data =( hcba_replac_srch_with_booking.unionAll(hcba_srch_data_cleaned)
                    .unionAll(hcba_booking_data_cleaned)
                    .repartition(1)
                    .cache())

# Subset the HCBA Table for Populating Average Weekly Traffic in Opti AD

In [23]:
# Only current years data
hcba_opti = (hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-01-01'").filter("ACTIVITY_DATE <= '2016-06-18'")# valid 1
                   .withColumn('WEEK',weekofyear("ACTIVITY_DATE"))) 

hcba_opti_agg = hcba_opti.groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE')

hcba_opti_agg_total_traffic = (hcba_opti_agg.agg({'TRAFFIC':'sum','BOOKING':'sum','GROSS_PROFIT':'sum','COST':'sum'})
                                .withColumnRenamed('sum(TRAFFIC)','TRAFFIC')
                                .withColumnRenamed('sum(BOOKING)','BOOKING')
                                .withColumnRenamed('sum(GROSS_PROFIT)','GROSS_PROFIT')
                                .withColumnRenamed('sum(COST)','COST')
                                .repartition(1))

hcba_opti_agg_number_weeks =( hcba_opti_agg.agg(func.countDistinct('WEEK'))
                            .withColumnRenamed('count(WEEK)','NO_WEEKS')
                            .repartition(1))

hcba_opti_with_cols_traffic_weeks =(hcba_opti_agg_total_traffic
                                    .join(hcba_opti_agg_number_weeks,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'inner')
                                    .filter("DEVICE_TYPE = 'DESKTOP'")
                                    .repartition(1))

hcba_opti_final1 = (hcba_opti_with_cols_traffic_weeks.withColumn('AVERAGE_WEEKLY_TRAFFIC',
                    hcba_opti_with_cols_traffic_weeks.TRAFFIC/hcba_opti_with_cols_traffic_weeks.NO_WEEKS)
                    .withColumn('CPC_HCBA',
                    hcba_opti_with_cols_traffic_weeks.COST/hcba_opti_with_cols_traffic_weeks.TRAFFIC)
                   .groupBy('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE')
                   .agg({'AVERAGE_WEEKLY_TRAFFIC':'avg','CPC_HCBA':'avg'})
                   .withColumnRenamed("avg(AVERAGE_WEEKLY_TRAFFIC)","AVERAGE_WEEKLY_TRAFFIC")
                   .withColumnRenamed("avg(CPC_HCBA)","CPC_HCBA")
                   .filter("DEVICE_TYPE = 'DESKTOP'")
                   .drop("DEVICE_TYPE"))
hcba_opti_final = (hcba_opti_final1
                  .join(hcba_opti_with_cols_traffic_weeks,['HOTEL_ID','LOS_BUCKET','BW_BUCKET'],'inner')
                  .drop("NO_WEEKS")
                  .repartition(1)
                  .cache())

# Creating the Training AD for RF using HCBA 

In [24]:
# splitting HCBA for training and validation
hcba_rf_union_train1 = hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-05-29'").filter("ACTIVITY_DATE <= '2016-06-18'")
hcba_rf_union_train2 = hcba_cleaned_data.filter("ACTIVITY_DATE < '2016-01-01'")
hcba_rf_union_train = hcba_rf_union_train1.unionAll(hcba_rf_union_train2)
hcba_rf_union_valid = hcba_cleaned_data.filter("ACTIVITY_DATE >= '2016-06-19'")

In [25]:
## Join HCBA and MBAP Table
rf_training_data = (hcba_rf_union_train.join(hotel_dim_final,"HOTEL_ID","inner")
                       .repartition(1)
                       .cache())

rf_validation_data = (hcba_rf_union_valid.join(hotel_dim_final,"HOTEL_ID","inner")
                       .repartition(1)
                       .cache())


# Importing EPB Master table (If Net Revenue then TOP_HOTEL_PRICE_AGG)

In [26]:
###change the table that needs to imported after finalising the approach
opti_search_query_universe = (sqlContext.read.parquet("s3n://ewe-meta-top/sq_univ_3Apr_23Apr")
                              .select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","EPB")
                              .repartition(4,"HOTEL_ID","LOS_BUCKET","BW_BUCKET")
                              .cache())

# Creating Optimization AD

In [27]:
# Joining optimization universe with EPB field and HCBA for average weekly traffic and hotel_dim for hotel dimensions
opti_search_query_universe_final = (opti_search_query_universe[opti_search_query_universe.EPB.isNotNull()]
                                                  .join(hcba_opti_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],'left_outer')
                                                  .join(hotel_dim_final,"HOTEL_ID","inner")
                                                  .repartition(4)
                                                  .cache()
                                                  )

# Random Forest Model Generation

In [28]:
from pyspark.mllib.linalg import SparseVector, VectorUDT
import numpy
from pyspark.ml.feature import *
from pyspark.mllib.linalg import *
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

In [29]:
def all_regions(CITY_BUCKET,region,MARKET_BUCKET,REGION_BUCKET,name):
    
    if name == 'CANADA':
        region_level = CITY_BUCKET
    elif name == 'US':
        region_level = region
    elif name in ['Mexico', 'CARIBBEAN', 'North America']:
        region_level = MARKET_BUCKET
    else:
        region_level = REGION_BUCKET
        
    return region_level

all_region_cm_bkt = udf(all_regions,StringType())
    

    

## Subset data for training and optimization of Canada Model

In [30]:
training_ad = (rf_training_data
               .filter("DEVICE_TYPE='DESKTOP'")
               .withColumn('REGION_LEVEL_BUCKET',all_region_cm_bkt("CITY_BUCKET","region","MARKET_BUCKET","REGION_BUCKET","MODEL_NAME")))

optimization_test = (opti_search_query_universe_final
                     .withColumn('REGION_LEVEL_BUCKET',all_region_cm_bkt("CITY_BUCKET","region","MARKET_BUCKET","REGION_BUCKET","MODEL_NAME")))

In [31]:
# adding code for valdiation
validation_ad = (rf_validation_data
               .filter("DEVICE_TYPE='DESKTOP'")
               .withColumn('REGION_LEVEL_BUCKET',all_region_cm_bkt("CITY_BUCKET","region","MARKET_BUCKET","REGION_BUCKET","MODEL_NAME")))

In [32]:
# for training the data
# Aggregating the data to the required level
col_agg = ["BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET", "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET"]
training_ad_T = (training_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))
validation_ad_T = (validation_ad.groupBy(col_agg)
                      .sum("TRAFFIC", "BOOKING","COST","GROSS_PROFIT")
                      .withColumnRenamed('sum(TRAFFIC)', 'TRAFFIC')
                      .withColumnRenamed('sum(BOOKING)', 'BOOKING')
                      .withColumnRenamed('sum(COST)', 'COST')
                      .withColumnRenamed('sum(GROSS_PROFIT)', 'GROSS_PROFIT'))

In [33]:
# Filtering out the TRAFFIC (Outliers) from data
training_ad_T1 = (training_ad_T[~((training_ad_T.TRAFFIC <= 4) & (training_ad_T.BOOKING > 0))]
                               [~((training_ad_T.TRAFFIC == 5) & (training_ad_T.BOOKING == 2))]
                               [~((training_ad_T.TRAFFIC == 6) & (training_ad_T.BOOKING == 2))])

In [34]:
col_required_rf = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC",
                   "EPB","TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [35]:
training_ad_T2 = (training_ad_T1.withColumn("chk_data",lit("train"))
                                .withColumn('MODEL_NAME',lit("all"))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_rf))

In [36]:
optimization_test2 = (optimization_test.withColumn('TRAFFIC',lit(0))
                                      .withColumn('BOOKING',lit(0))
                                      .withColumn('GROSS_PROFIT',lit(0.00))
                                      .withColumn('COST',lit(0.00))
                                      .withColumn("DEVICE_TYPE",lit("DESKTOP"))
                                      .withColumn("chk_data",lit("test"))
                                      .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                      .select(col_required_rf))

In [37]:
training_for_test = training_ad_T 
validation_for_test = validation_ad_T 

In [38]:
col_required_results = ["MODEL_NAME","HOTEL_ID","LOS_BUCKET","BW_BUCKET","DEVICE_TYPE","STAR_RATING_BUCKET",
                   "HOTEL_TYPE_BUCKET","REGION_LEVEL_BUCKET","AVERAGE_WEEKLY_TRAFFIC","EPB",
                        "TRAFFIC","BOOKING","chk_data","CPC_HCBA","GROSS_PROFIT","COST",'ACTIVITY_DATE']

In [39]:
# for testing on training data
training_for_test1 = (training_for_test.withColumn("chk_data",lit("train-testing"))
                                .withColumn('MODEL_NAME',lit("all"))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [40]:
# for testing on validation data
validation_for_test1 = (validation_for_test.withColumn("chk_data",lit("validation-testing"))
                                .withColumn('MODEL_NAME',lit("all"))
                                .withColumn('HOTEL_ID',lit("NA"))
                                .withColumn('DEVICE_TYPE',lit("DESKTOP"))
                                .withColumn('AVERAGE_WEEKLY_TRAFFIC',lit(0.0))
                                .withColumn('EPB',lit(0.00))
                                .withColumn('REGION_BUCKET',lit("NA"))
                                .withColumn('CPC_HCBA',lit(0.0))
                                .withColumn('ACTIVITY_DATE',lit('2001-01-01')) # column necessary for testing results
                                .select(col_required_results))

In [41]:
test_train_data = (optimization_test2.unionAll(training_ad_T2)
                                    .unionAll(training_for_test1)
                                    .unionAll(validation_for_test1)
                                    .repartition(16)
                                    .cache())

In [42]:
# STRING INDEXING: Converting the data type of required columns to factor
def indexStringColumns(df, cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col, outputCol=col + "-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf

In [43]:
cols_rf_operations = {"BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET"}
dfnumeric = indexStringColumns(test_train_data, cols_rf_operations)

In [44]:
# One HOT Encoding
def oneHotEncodeColumns(df, cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(
            inputCol=col + "-num", outputCol=col + "-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col + "-num")
    return tempdf

In [45]:
test_train_data_class = oneHotEncodeColumns(dfnumeric, cols_rf_operations)

In [46]:
cols_rf_model = ["BW_BUCKET","LOS_BUCKET","HOTEL_TYPE_BUCKET","STAR_RATING_BUCKET","REGION_LEVEL_BUCKET", 
                "TRAFFIC","BOOKING","STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot", 
                "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot"]

In [47]:
training_ad_T1_Class = (test_train_data_class.filter(test_train_data_class.chk_data=="train")
                        .drop("chk_data")
                        .select(cols_rf_model))


In [48]:
# Transforming the training dataset
# Defining function for the same
def genRows(row):
    TRAFFIC = row[5]
    BOOKING = row[6]
    BOOKINGcounter = 0
    rowList = []

    for i in range(TRAFFIC):
        row1 = row[:]
        row1 = list(row1)
        row1[5] = 1
        row1 = tuple(row1)
        if BOOKINGcounter < int(BOOKING):
            row1 = list(row1)
            row1[6] = 1
            row1 = tuple(row1)
        else:
            row1 = list(row1)
            row1[6] = 0
            row1 = tuple(row1)
        BOOKINGcounter += 1
        rowList.append(row1)
    return rowList

In [49]:
training_ad_T1_Class_rdd = training_ad_T1_Class.rdd
training_ad_T1_Class_rdd = training_ad_T1_Class_rdd.flatMap(lambda x: genRows(x))

In [50]:
# Reconverting to dataframe
from pyspark.mllib.linalg import SparseVector, VectorUDT

schema = StructType([StructField('BW_BUCKET', StringType(), True),
                     StructField('LOS_BUCKET', StringType(), True),
                     StructField('HOTEL_TYPE_BUCKET', StringType(), True),
                     StructField('STAR_RATING_BUCKET', StringType(), True),
                     StructField('REGION_LEVEL_BUCKET',StringType(), True),
                     StructField('TRAFFIC', IntegerType(), True),
                     StructField('BOOKING', IntegerType(), True),
                     StructField('HOTEL_TYPE_BUCKET-onehot',VectorUDT(), True),
                     StructField('BW_BUCKET-onehot',  VectorUDT(), True),
                     StructField('REGION_LEVEL_BUCKET-onehot',VectorUDT(), True),
                     StructField('LOS_BUCKET-onehot',  VectorUDT(), True),
                     StructField('STAR_RATING_BUCKET-onehot',  VectorUDT(), True)])

training_ad_T1_Class_df = sqlContext.createDataFrame(training_ad_T1_Class_rdd, schema)

In [51]:
# Converting label to double datatype
toDouble = udf(lambda x: float(x * 1.00), DoubleType())

training_ad_T1_Class_df = training_ad_T1_Class_df.withColumn('BOOKING', toDouble(training_ad_T1_Class_df.BOOKING))

In [52]:
# Taking only relevant columns as features
colList = training_ad_T1_Class_df.columns
colList.remove('BOOKING')
colList.remove('TRAFFIC')
colList.remove('BW_BUCKET')
colList.remove('LOS_BUCKET')
colList.remove('HOTEL_TYPE_BUCKET')
colList.remove('STAR_RATING_BUCKET')
colList.remove('REGION_LEVEL_BUCKET')

vecAssembler = VectorAssembler(inputCols=colList, outputCol="features")

cols_train_ad_rf = ["BOOKING", "TRAFFIC", "BW_BUCKET", "LOS_BUCKET", "HOTEL_TYPE_BUCKET",
                    "STAR_RATING_BUCKET", "REGION_LEVEL_BUCKET", "features"]

train_ad = (vecAssembler.transform(training_ad_T1_Class_df)
                        .select(cols_train_ad_rf)
                        .withColumnRenamed("BOOKING", "label")
                        .repartition(40)
                        .cache())

In [53]:
col_remove = ["STAR_RATING_BUCKET-onehot","REGION_LEVEL_BUCKET-onehot","BW_BUCKET-onehot",
              "HOTEL_TYPE_BUCKET-onehot","LOS_BUCKET-onehot"]

optimization_onehot = (test_train_data_class.filter(test_train_data_class.chk_data=="test")
                                            .drop("chk_data"))

vecAssembler = VectorAssembler(inputCols=colList, outputCol="indexedFeatures")

optimization_df_onehot = vecAssembler.transform(optimization_onehot)
                          
optimization_df = (optimization_df_onehot.select([col for col in optimization_df_onehot.columns if col not in col_remove])
                  .repartition(100)
                  .cache())

In [54]:
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(train_ad)

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(train_ad)

rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures", featureSubsetStrategy="all",
                           numTrees=1000, impurity="variance", maxBins=32,maxDepth=4)

pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

model = pipeline.fit(train_ad)

RF_Model = model.stages[2]

In [55]:
optimization_df_predict = (RF_Model.transform(optimization_df)
                           .drop("indexedFeatures")
                           .withColumnRenamed("prediction", "CONVERSION_RATE")
                           .repartition(100))

In [56]:
region_df_for_opti = (optimization_df_predict
                      .cache())

# Optimization algorithm

In [57]:
##Pulp package for Linear Programming Optimization algorithm
import pulp
from pulp import *
import pandas as pd
import numpy as np

In [58]:
#if conversion rate is less then 0.0005 then taking it as 0
def predicted(predicted):
    if predicted < 0.0005:
        predicted = 0.0
    else:
        predicted
    return predicted

##If Net revenue is negative then making it positive 
def net_revenue(net_revenue):
    if net_revenue < 0.0 :
        net_revenue = net_revenue*(-1)
    else:
        net_revenue
    return net_revenue

##creating unique id for each search query
def var_name(id):
    return "T_" + str(id)


In [59]:
# ## UDF for Objective Function and Lower Limit for LP Solve

##Creating the objective function at search query level, combining it while running lp_solve algorithm
def objective_fun(net_revenue,predicted):
    return net_revenue * predicted

##Lower limit for the average weekly Traffic
def lower_limit_fun(average_weekly_traffic):
    limit = 0.2*average_weekly_traffic
    return limit


##registering the above defined functions as uoptimization_rf
objective_fun_udf = udf(objective_fun, DoubleType())
lower_limit_fun_udf = udf(lower_limit_fun, DoubleType())
var_name_udf = udf(var_name,StringType())
net_revenue_udf = udf(net_revenue, DoubleType())
predicted_udf = udf(predicted , DoubleType())


In [60]:

#Filtering rows with Traffic > 0 
optimization_rf_algo = region_df_for_opti[~region_df_for_opti.AVERAGE_WEEKLY_TRAFFIC.isNull()]

#Applying Prediction UDF for rounding CR less than 0.0005
region_df_algo_treated = (optimization_rf_algo.withColumn("predicted_new",predicted_udf("CONVERSION_RATE"))
                            .withColumn("net_revenue_new",net_revenue_udf("EPB")))

region_df_algo_final= (region_df_algo_treated.withColumn("objective_fun",objective_fun_udf("net_revenue_new" ,"predicted_new"))
                            .withColumn("Lower_Limit", lower_limit_fun_udf("AVERAGE_WEEKLY_TRAFFIC"))
                            .withColumn("id", monotonicallyIncreasingId())
                            .withColumn("var_name" ,var_name_udf("id"))
                            .rdd
                            .repartition(1)
                            .cache())

region_count = region_df_algo_final.count()


In [61]:
##list of variuable names for opti algo
t_list = region_df_algo_final.map(lambda x  : x.var_name).collect()

##objective function for lp solve
objective_list =  region_df_algo_final.map(lambda x  : x.objective_fun).collect()
final_objective = {t_list[i]: objective_list[i] for i in range(0, region_count)}

##lower and upper limit 
lower_limit_list =  region_df_algo_final.map(lambda x  : x.Lower_Limit).collect()
upper_limit_list =  region_df_algo_final.map(lambda x  : x.AVERAGE_WEEKLY_TRAFFIC).collect()
lower_limit_final = {t_list[i]: lower_limit_list[i] for i in range(0, region_count)}
upper_limit_final = {t_list[i]: upper_limit_list[i] for i in range(0, region_count)}


In [62]:
## Optimization Algorithm
max_gp  = (region_df_algo_final
            .map(lambda x : x.objective_fun*x.AVERAGE_WEEKLY_TRAFFIC)
            .reduce(lambda x,y : x+y))
max_cpc = (region_df_algo_final
        .map(lambda x : x.CPC_HCBA*x.AVERAGE_WEEKLY_TRAFFIC)
        .reduce(lambda x,y : x+y))
min_cpc = (region_df_algo_final
        .map(lambda x : x.CPC_HCBA*x.Lower_Limit)
        .reduce(lambda x,y : x+y))

eff_max_GP = max_cpc/max_gp
lower_eff= min_cpc/max_gp
upper_eff=1


In [63]:
lp_status = "Start"
region_name = region_df_algo_final.first()[0]

cnt = lower_eff
while ((lp_status != "Optimal") & (cnt<eff_max_GP)):

    final_constraint = region_df_algo_final.map(lambda x: (x.CPC_HCBA - cnt*x.objective_fun)).collect()

    constraint_eff = {t_list[i]: final_constraint[i] for i in range(0, region_count)}

    prob = LpProblem("The Optimization Problem", LpMaximize)

    lp_var = {}
    for variable in t_list:
        variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
        lp_var[str(variable)] = variable

    prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])

    prob += lpSum([constraint_eff[i]*lp_var[i] for i in t_list]) == 0


    prob.solve()

    lp_status = LpStatus[prob.status]

    if (lp_status == "Optimal"):
        min_eff_optimal = cnt
        
    print(cnt)
    cnt+=0.1


0.31600331516256847
0.4160033151625685
0.5160033151625685
0.6160033151625685


In [64]:
region_df_algo_final_spark = sqlContext.createDataFrame(region_df_algo_final) 


hcba_updatedcols = (hcba_opti_final.select('HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE','CPC_HCBA','AVERAGE_WEEKLY_TRAFFIC')
                                .withColumnRenamed('AVERAGE_WEEKLY_TRAFFIC','AVERAGE_WEEKLY_TRAFFIC-usedintraining')
                                .withColumnRenamed('CPC_HCBA','CPC_HCBA-usedintraining'))


In [65]:
def df_create(x):
    mydictionary = x['var_list']
    
    Model_Name = x['region_name'][0]
    
    var_list = []
    for key in mydictionary:
        var_list.append((key, mydictionary[key],Model_Name))
          
    df = pd.DataFrame(var_list, columns=['var_id', 'var_value','Model'])
    df_final = sqlContext.createDataFrame(df)
    
    return df_final 


def threshold(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (1-z)

threshold_udf = udf(threshold,DoubleType())

def traffic_allow(conv_rate,avg_weekly_traffic):
    z = conv_rate/avg_weekly_traffic 
    return (z)

traffic_allow_udf = udf(traffic_allow,DoubleType())



In [66]:
cnt = min_eff_optimal
rule_look_up = pd.DataFrame()
counter = 1
while (cnt<eff_max_GP):

    final_constraint = region_df_algo_final.map(lambda x: (x.CPC_HCBA - cnt*x.objective_fun)).collect()

    constraint_eff = {t_list[i]: final_constraint[i] for i in range(0, region_count)}

    prob = LpProblem("The Optimization Problem", LpMaximize)

    lp_var = {}
    for variable in t_list:
        variable = LpVariable(variable,lower_limit_final[variable],upper_limit_final[variable])
        lp_var[str(variable)] = variable

    prob += lpSum([final_objective[i]*lp_var[i] for i in t_list])

    prob += lpSum([constraint_eff[i]*lp_var[i] for i in t_list]) == 0
    prob.solve()
    lp_status = LpStatus[prob.status]

    if(lp_status == "Optimal"):
        
        cnt += 0.1
        
        var_values = {"start":0}
        for v in prob.variables():
            var_values.update({v.name[2:]:v.varValue})

        results={"region_name":region_name,"lp_status":LpStatus[prob.status], "gp":value(prob.objective),"var_list":var_values}
        
        final_results= [results]
        
        
        field = [StructField("var_id", StringType(), True)
                 ,StructField("var_value", DoubleType(), True)
                 ,StructField("Model", StringType(), True)]

        schema = StructType(field)
        optimization_df_algo1 = sqlContext.createDataFrame(sc.emptyRDD(), schema)

        for i in range(len(final_results)):
            optimization_df_algo1 = optimization_df_algo1.unionAll(df_create(final_results[i]))

        optimization_df_algo1 = optimization_df_algo1.cache()
        
        optimization_df_algo1 = optimization_df_algo1.withColumn("id",optimization_df_algo1["var_id"].cast(LongType()))
        optimization_df_algo2 = (region_df_algo_final_spark
                                .select([col for col in region_df_algo_final_spark.columns
                                    if col not in {"predicted_new","net_revenue_new","objective_fun","Lower_Limit","var_name","part_id"}])
                                 .join(optimization_df_algo1,"id","inner"))

        optimization_df_algo3 = optimization_df_algo2.withColumn("threshold",threshold_udf("var_value","AVERAGE_WEEKLY_TRAFFIC")).                        withColumn("traffic_allow",traffic_allow_udf("var_value","AVERAGE_WEEKLY_TRAFFIC"))

        optimization_df_algo3 = optimization_df_algo3.select([col for col in optimization_df_algo3.columns if col not in {"id","var_id","Model"}])
        optimization_df_algo3 = optimization_df_algo3.withColumnRenamed("var_value","Lp_solution")
        optimization_df_algo_final = optimization_df_algo3.select("HOTEL_ID","LOS_BUCKET","BW_BUCKET","threshold","EPB","traffic_allow")
        
        validation_df_spark = (validation_ad
                              .join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer')
                              .filter(validation_ad.DEVICE_TYPE=='DESKTOP')
                              )

        common_queries = validation_df_spark.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")
        
        valid_df = common_queries.withColumn("throttle_traffic",(common_queries.traffic_allow*common_queries.TRAFFIC))

        valid_df = valid_df.withColumn("throttle_cost",((valid_df.COST*valid_df.throttle_traffic)/valid_df.TRAFFIC))
        valid_df = valid_df.withColumn("average_booking",(valid_df.BOOKING*valid_df.traffic_allow))
        valid_df = valid_df.withColumn("average_earning",((valid_df.GROSS_PROFIT*valid_df.average_booking)/valid_df.BOOKING))

       
        if counter == 1: #change it to the first number from which starting to run
            valid_actual=valid_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"})

            valid_throttle = (valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                                .withColumn("Efficiency",lit(cnt)))

        else :
            valid_throttle = valid_throttle.unionAll((valid_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                                                     .withColumn("Efficiency",lit(cnt))))


        # using training data second
        training_df_spark =( training_ad.join(hcba_updatedcols,['HOTEL_ID','LOS_BUCKET','BW_BUCKET','DEVICE_TYPE'],'left_outer')
                               .filter(training_ad.DEVICE_TYPE=='DESKTOP')
                                .filter(training_ad.ACTIVITY_DATE>'2016-01-01'))
        

        common_queries_train = training_df_spark.join(optimization_df_algo_final,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"inner")

        train_df = common_queries_train.withColumn("throttle_traffic",(common_queries_train.traffic_allow*common_queries_train.TRAFFIC))

        train_df = train_df.withColumn("throttle_cost",((train_df.COST*train_df.throttle_traffic)/train_df.TRAFFIC))
        train_df = train_df.withColumn("average_booking",(train_df.BOOKING*train_df.traffic_allow))
        train_df = train_df.withColumn("average_earning",((train_df.GROSS_PROFIT*train_df.average_booking)/train_df.BOOKING))



        if counter == 1:#change it to the first number from which starting to run
            train_actual = train_df.agg({"TRAFFIC":"sum","COST":"sum","GROSS_PROFIT":"sum","BOOKING":"sum"})


            train_throttle = (train_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                                .withColumn("Efficiency",lit(cnt))) 

        else :
            train_throttle = train_throttle.unionAll((train_df.agg({"throttle_traffic":"sum","throttle_cost":"sum","average_earning":"sum","average_booking":"sum"})
                                                    .withColumn("Efficiency",lit(cnt))))


        counter += 1
        print(counter)



            


2
3
4
5
6
7
8
9
10
11


In [67]:
valid_throttle.show(50)

+---------------------+------------------+--------------------+--------------------+------------------+
|sum(throttle_traffic)|sum(throttle_cost)|sum(average_earning)|sum(average_booking)|        Efficiency|
+---------------------+------------------+--------------------+--------------------+------------------+
|   23010.599998152768| 22279.13280691579|    21219.6950158611|  439.80000007072357|0.7160033151625684|
|    26980.19999699317|26849.047920869274|   27135.83589617541|   523.8000001144883|0.8160033151625684|
|   30530.665097719768|31268.922382134046|   31444.65350782799|   612.6000001629774|0.9160033151625684|
|   33520.199990873916| 35139.21819502211|   34982.22502637425|    697.400000034378|1.0160033151625685|
|    36417.79998996136|38938.471119949616|  37904.806937923706|    772.599999984854|1.1160033151625686|
|    39106.19722333959|  42626.9095459755|   41267.05006332285|    845.400000086523|1.2160033151625687|
|     41625.7999876241| 46193.73257987258|  45336.443263801346| 

In [68]:
valid_actual.show()

+------------+----------------+-----------------+------------+
|sum(TRAFFIC)|       sum(COST)|sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+----------------+-----------------+------------+
|       49009|57742.1083840066|       53865.2947|        1279|
+------------+----------------+-----------------+------------+



In [69]:
train_throttle.show(50)

+---------------------+------------------+--------------------+--------------------+------------------+
|sum(throttle_traffic)|sum(throttle_cost)|sum(average_earning)|sum(average_booking)|        Efficiency|
+---------------------+------------------+--------------------+--------------------+------------------+
|    163015.0000061844|112278.02273390436|  156153.92468879657|   2816.600000122568|0.7160033151625684|
|   186827.00000016217|139711.25648209397|  183480.76966167742|   3284.600000158361|0.8160033151625684|
|   205608.92551156325| 165434.3422048431|  210387.76901614186|   3764.599999569979|0.9160033151625684|
|   221322.91206552475|190862.84271532044|  232162.53965508429|  4214.9999995196495|1.0160033151625685|
|   235206.99997880825|215799.46738398008|  253843.13140152712|   4677.399999185933|1.1160033151625686|
|   248142.19996936448|241794.56609322256|   277935.4908969137|   5170.199998514017|1.2160033151625687|
|    259968.5999654144| 267273.7053175886|  296611.39809363073| 

In [70]:
train_actual.show()

+------------+------------------+-----------------+------------+
|sum(TRAFFIC)|         sum(COST)|sum(GROSS_PROFIT)|sum(BOOKING)|
+------------+------------------+-----------------+------------+
|      293119|358338.51055216725|351147.2390000001|        7423|
+------------+------------------+-----------------+------------+

